In [8]:
# !pip install pyniryo2
# !pip install pyniryo
# !pip install pynput

In [9]:
# https://github.com/NiryoRobotics/pyniryo2
# https://docs.niryo.com/dev/pyniryo2/v1.0.0/en/index.html

In [10]:
from pyniryo2 import *
import pyniryo
# from matplotlib import pyplot as plt
import cv2 as cv
import time

# https://pynput.readthedocs.io/en/latest/keyboard.html
from pynput import keyboard

In [11]:
robot_ip_address = "169.254.200.200"

robot = NiryoRobot(robot_ip_address)
robot.arm.calibrate_auto()
robot.tool.update_tool()

# ETH
ros = robot.client
vision = Vision(ros)

In [12]:
J1_MAX = 2.9
J1_MIN = -2.9

J2_MAX = 0.61
J2_MIN = -1.83

J3_MAX = 1.57
J3_MIN = -1.34

J4_MAX = 2.09
J4_MIN = -2.09

J5_MAX = 1.54
J5_MIN = -1.34

J6_MAX = 2.53
J6_MIN = -2.529

In [13]:
# Movement:
# Joint 1: Base
# A/D
# Joint 2: Shoulder
# W/S
# Joint 3: Elbow
# I/K
# Joint 4: Forearm
# J/L
# Joint 5: Wrist
# Arrow Up/Down
# Joint 6: Hand
# Arrow Left/Right
# Gripper
# +/-

In [14]:
joints = robot.arm.get_joints()
j1, j2, j3, j4, j5, j6 = joints
j1_is_moving = False
j2_is_moving = False
j3_is_moving = False
j4_is_moving = False
j5_is_moving = False
j6_is_moving = False

def move_callback(result):
    if result == None:
        return
    if result["status"] == -4:
        print("Done")
    print(result)

def stop_callback(result):
    print("stop callback")
    print(result)

def error_callback(result):
    print("stop")

    

def refreshJoints():
    global j1, j2, j3, j4, j5, j6
    joints = robot.arm.get_joints()
    j1, j2, j3, j4, j5, j6 = joints
    return joints

def on_press(key):
    global j1, j2, j3, j4, j5, j6
    global j1_is_moving, j2_is_moving, j3_is_moving, j4_is_moving, j5_is_moving, j6_is_moving
    
    try:
        if isinstance(key, keyboard.KeyCode):
            if not j1_is_moving:
                if key.char == 'a':
                    j1_is_moving = True
                    j1 = J1_MAX
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
                if key.char == "d":
                    j1_is_moving = True
                    j1 = J1_MIN
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
            if not j2_is_moving:
                if key.char == "w":
                    j2_is_moving = True
                    j2 = J2_MIN
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
                if key.char == "s":
                    j2_is_moving = True
                    j2 = J2_MAX
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
            if not j3_is_moving:
                if key.char == 'i':
                    j3_is_moving = True
                    j3 = J3_MIN
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
                if key.char == "k":
                    j3_is_moving = True
                    j3 += J3_MAX
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
            if not j4_is_moving:
                if key.char == "j":
                    j4_is_moving = True
                    j4 = J4_MIN
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
                if key.char == "l":
                    j4_is_moving = True
                    j4 = J4_MAX
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
        else:
            if not j5_is_moving:
                if key == keyboard.Key.up:
                    j5_is_moving = True
                    j5 = J5_MIN
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
                if key == keyboard.Key.down:
                    j5_is_moving = True
                    j5 = J5_MAX
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)

            if not j6_is_moving:
                if key == keyboard.Key.left:
                    j6_is_moving = True
                    j6 = J6_MIN
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
                if key == keyboard.Key.right:
                    j6_is_moving = True
                    j6 = J6_MAX
                    robot.arm.move_joints([j1, j2, j3, j4, j5, j6], move_callback)
    except AttributeError:
        print('special key {0} pressed'.format(
            key))

def move_to_last_position():
    # time.sleep(0.5)
    # robot.arm.move_joints(last_position)
    return

def on_release(key):
    global j1_is_moving, j2_is_moving, j3_is_moving, j4_is_moving, j5_is_moving, j6_is_moving

    if isinstance(key, keyboard.KeyCode):
        if key.char == "a" or key.char == "d":
            print("Release A or D", j1)
            j1_is_moving = False
            robot.arm.stop_move()
            move_to_last_position()
        if key.char == "w" or key.char == "s":
            print("Release W or S", j2)
            j2_is_moving = False
            robot.arm.stop_move()
            move_to_last_position()
        if key.char == "i" or key.char == "k":
            print("Release I or K", j3)
            j3_is_moving = False
            robot.arm.stop_move()
            move_to_last_position()
        if key.char == "j" or key.char == "l":
            print("Release J or L", j4)
            j4_is_moving = False
            robot.arm.stop_move()
            move_to_last_position()

        if key.char == '+':
            robot.tool.grasp_with_tool()
        if key.char == "-":
            robot.tool.release_with_tool()     
    else:
        if key == keyboard.Key.up or key == keyboard.Key.down:
            print("Release arrow up or down", j5)
            j5_is_moving = False
            robot.arm.stop_move()
            move_to_last_position()
        if key == keyboard.Key.left or key == keyboard.Key.right:
            print("Release arrow left or right", j6)
            j6_is_moving = False
            robot.arm.stop_move()
            move_to_last_position()

        if key == keyboard.Key.esc:
            # Stop listener
            return False

listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release
    )

def cleanup():
    print("CLEANUP")
    cv.destroyAllWindows()
    listener.stop()
    robot.arm.stop_move()

cleanup()
listener.start()

last_position = None
robot.arm.joints_state.unsubscribe()
robot.arm.joints_state.subscribe(lambda message: globals().update(last_position=message.position))

print("START")
while listener.running:
    try:
        joints = refreshJoints()
        # continue
        img_compressed = vision.get_img_compressed()
        camera_info = vision.get_camera_intrinsics()

        img = pyniryo.uncompress_image(img_compressed)
        img_uncompressed = img
        img = pyniryo.undistort_image(img, camera_info.intrinsics, camera_info.distortion)

        #plt.imshow(img)
        cv.imshow("stream - raw", img_uncompressed)
        cv.imshow("stream - undistorted", img)
        cv.waitKey(1)
    except:
        print("oh no")
        cleanup()
cleanup()

CLEANUP
START
Release W or S -0.3641101434400832
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release A or D 1.0143965211048505
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release A or D 0.4238888033020971
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release A or D -0.29598271865847625
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release arrow up or down -0.7640150859568422
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release arrow up or down -0.6627723539563899
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release J or L -0.5567423724126948
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
Release J or L -1.2761793619310606
Done
{'status': -4, 'message': 'Command has been successfully stopped'}
{'status': -52, 'message': 'joint_3 not in range (-1.3400637996813345, 1.5700981950942021)'}
Release I or K 0.2